In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import math

In [ ]:
housing = fetch_california_housing()

In [ ]:
print(type(housing['data']))
print(type(housing['target']))

In [ ]:
inputs = housing.data
output = housing.target 

#housing.target is a 1D araay but we need to reshape it to a column vector to compute theta.
#Numpy has reshape function accepts -1 (meaning "unspecificied") for one of the dimensions: that dimension will be
#computed vased on the array's length and the remaining dimensions.
output=output.reshape(-1,1)

In [ ]:
m, n = housing.data.shape
#m is number of observations
#n is number of variables (features)

In [ ]:
housing_data_plus_bias = np.c_[np.ones((m,1)), inputs]

# Using Normal Equations

In [ ]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(output, dtype = tf.float32, name = "y")

In [ ]:
#\hat{\theta} = ((X^{T} \times X)^{-1}) \times X^{T} \times y --> normal equations
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(tf.transpose(X), X)) , tf.transpose(X)) ,y)

In [ ]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [ ]:
theta_value

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

# Implementing Gradient Descent

In [ ]:
scaled_housing_data = StandardScaler().fit_transform(inputs)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

# Manually computing the gradients

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(output, dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name ="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="MSE")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        sess.run(training_op)
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
    best_theta = theta.eval()

In [ ]:
best_theta

#  Using autodiff

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(output, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients  = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

In [ ]:
print("Best theta:")
print(best_theta)

# Using a GradientDescentOptimizer

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(output, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

In [ ]:
print("Best theta:")
print(best_theta)

# Using a momentum optimizer

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(output, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

In [ ]:
print("Best theta:")
print(best_theta)

# Mini-batch Gradient Descent 

In [ ]:
scaled_housing_data = StandardScaler().fit_transform(inputs)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

batch_size = 500

def miniBatch(x, y, batchSize):
    numObs  = x.shape[0]
    batches = []
    batchNum = math.floor(numObs / batchSize)

    for i in range(batchNum - 1):
        xBatch = x[i * batchSize:(i + 1) * batchSize, :]
        yBatch = y[i * batchSize:(i + 1) * batchSize, :]
        batches.append((xBatch, yBatch))
    xBatch = x[batchNum * batchSize:, :]
    yBatch = y[batchNum * batchSize:, :]
    batches.append((xBatch, yBatch))
    return batches


n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n+1], name = "X")
y = tf.placeholder(tf.float32, shape = [None, 1], name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        miniBatches = miniBatch(scaled_housing_data_plus_bias, output, batch_size)
        batchNum = len(miniBatches)
        for batch in miniBatches:
            xBatch = batch[0]
            yBatch = batch[1]
            sess.run(training_op, feed_dict={X: xBatch, y: yBatch})

    best_theta = theta.eval()

In [ ]:
best_theta

# Saving and Restoring Model

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(output, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)

saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        if epoch % 100 == 0: #Checkpoint at every 100 epochs
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

In [ ]:
best_theta

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() 

In [ ]:
 best_theta_restored

In [ ]:
#How do I find the variable names and values that are saved in a checkpoint?

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='/tmp/my_model_final.ckpt', tensor_name='', all_tensors=True, all_tensor_names=True)

#file_name: not a physical file, just the prefix of filenames
#If no tensor_name is provided, prints the tensor names and shapes in the checkpoint file. If tensor_name is provided, prints the content of the tensor.(inspect_checkpoint.py)
#If all_tensor_names is True, Prints all the tensor names
#If all_tensor is 'True`, Prints all the tensor names and the corresponding content.
#all_tensor and all_tensor_names will override tensor_name

In [ ]:
#Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(best_theta, best_theta_restored)

In [ ]:
saver = tf.train.Saver(var_list={"weights": theta})

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(output, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)

saver = tf.train.Saver(var_list={"weights": theta})
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        if epoch % 100 == 0: #Checkpoint at every 100 epochs
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

In [ ]:

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='/tmp/my_model_final.ckpt', tensor_name='', all_tensors=True, all_tensor_names=True)


In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    print('----------------------------------------------------')
    print(tf.global_variables())# print tensor variables
    print('----------------------------------------------------')
    print(tf.get_default_graph().get_tensor_by_name('theta:0'))
    print('----------------------------------------------------')
    print(tf.get_default_graph().get_operation_by_name("mse"))

In [ ]:
print(mse.op.name)
print(error.op.name)
print(theta.op.name)
print(mse.name)
print(error.name)
print(theta.name)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    print(sess.run(["mse:0"])) #print(sess.run(mse)) gives the same result
    print(sess.run(["theta:0"])) #print(sess.run(theta)) or print(theta.eval()) give the same result

# Name Scopes

In [ ]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [ ]:
with  tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [ ]:
optimizer  = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
scaled_housing_data = StandardScaler().fit_transform(inputs)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

In [ ]:
print(mse.op.name)
print(error.op.name)
print(theta.op.name)
print(mse.name)
print(error.name)
print(theta.name)

# Tensorboard visualization

In [ ]:
!pwd

In [ ]:
!ls -l tf_logs/run*

In [ ]:
#root log directory will be in the same folder where you run the code. Point tensorboard to the root log directory.
#If there is more than one log folder, choose which one you want to visualize, like:
#!tensorboard --logdir tf_logs/run-20180709100216

#This starts the TensorBoard web server, listening on port 6006 which is "goog" written upside down. 

#You also need to activate your virtualenv environment if you created one.
!source env/bin/activate
!tensorboard --logdir tf_logs/

#Next open a browser and go to http://0.0.0.0:6006/  (or http://localhost:6006/)

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph())

# Modularity

In [ ]:
import tensorflow as tf
sess=tf.InteractiveSession()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2") 

output = tf.add(relu1, relu2, name="output")


In [ ]:
def relu(X):
    w_shape = (int(X.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

import tensorflow as tf

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")
#tf.add_n adds all input tensors element-wise.

In [ ]:
file_writer = tf.summary.FileWriter("tf_logs/relu1", tf.get_default_graph())
file_writer.close()

In [ ]:
!ls -l tf_logs

In [ ]:
!tensorboard --logdir tf_logs/relu1
#http://0.0.0.0:6006/

In [ ]:
#Using name scopes, you can make graphs much clearer.

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

import tensorflow as tf

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")
#tf.add_n adds all input tensors element-wise.

file_writer = tf.summary.FileWriter("tf_logs/relu2", tf.get_default_graph())
!ls -l tf_logs
!tensorboard --logdir tf_logs/relu2
#http://0.0.0.0:6006/
file_writer.close()

# Sharing Variable

In [ ]:
import tensorflow as tf

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                    
        return tf.maximum(z, threshold, name="max")
    
n_features = 3
threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [ ]:
#printing all the tensors
graph = tf.get_default_graph()
list_of_tuples = [op.values() for op in graph.get_operations()]
list_of_tuples

In [ ]:
import tensorflow as tf 

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                    
        return tf.maximum(z, relu.threshold, name="max")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [ ]:
#printing all the tensors
graph = tf.get_default_graph()
list_of_tuples = [op.values() for op in graph.get_operations()]
list_of_tuples

In [ ]:
import tensorflow as tf

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))

In [ ]:
print(threshold.name)

In [ ]:
#printing all the tensors
graph = tf.get_default_graph()
list_of_tuples = [op.values() for op in graph.get_operations()]
list_of_tuples

In [ ]:
import tensorflow as tf

with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [ ]:
#printing all the tensors
graph = tf.get_default_graph()
list_of_tuples = [op.values() for op in graph.get_operations()]
list_of_tuples

In [ ]:
#1
import tensorflow as tf

with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [ ]:
#2
import tensorflow as tf

with tf.variable_scope("relu"):
    tf.get_variable_scope().reuse_variables()
    threshold = tf.get_variable("threshold")

In [ ]:
import tensorflow as tf

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [ ]:
file_writer = tf.summary.FileWriter("tf_logs/relu3", tf.get_default_graph())
file_writer.close()

In [ ]:
!ls -l tf_logs
!tensorboard --logdir tf_logs/relu3
#http://0.0.0.0:6006/

In [ ]:
import tensorflow as tf

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf_logs/relu4", tf.get_default_graph())
file_writer.close()

!ls -l tf_logs
!tensorboard --logdir tf_logs/relu3
#http://0.0.0.0:6006/

# More about variable_scope()

In [1]:
import tensorflow as tf

with tf.variable_scope("my_scope"):
    x0 = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    x1 = tf.Variable(0., name="x")
    x2 = tf.Variable(0., name="x")

with tf.variable_scope("my_scope", reuse=True):
    x3 = tf.get_variable("x")
    x4 = tf.Variable(0., name="x")

with tf.variable_scope("", default_name="", reuse=True):
    x5 = tf.get_variable("my_scope/x")

print("x0:", x0.op.name)
print("x1:", x1.op.name)
print("x2:", x2.op.name)
print("x3:", x3.op.name)
print("x4:", x4.op.name)
print("x5:", x5.op.name)
print(x0 is x3 and x3 is x5)

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True
